<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Finance_MULTIPLE_LLM_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai -q
!pip install anthropic -q
!pip install colab-env -q

In [4]:
import os
from typing import Dict, Any, List, Optional
import google.generativeai as genai
from anthropic import Anthropic
import colab_env # For Google Colab

# Import necessary libraries
try:
    import google.generativeai as genai
    from anthropic import Anthropic
    from anthropic.types import MessageParam
    # Conditional import for Colab's userdata
    try:
        from google.colab import userdata
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False
        print("Running outside Google Colab. 'userdata.get' will not be a available.") # Adjusted print statement
except ImportError:
    print("\n [CRITICAL ERROR] Please install necessary libraries:")
    print("pip install google-generativeai anthropic")
    print("Exiting as API functionality cannot be demonstrated without them.")
    # In a real application, you might raise an exception or handle this more gracefully
    # For this example, we'll allow execution but API calls will be mocked
    pass


# 1. Core Components (Interfaces and Implementations with API Management)
#
class BaseModel:
    """Base class for Language Models and Reasoning Models."""
    def __init__(self, name: str, identifier: str, api_client: Any):
        self.name = name
        self.identifier = identifier # The specific model string
        self.api_client = api_client # The actual API client instance

    def generate(self, prompt: str, **kwargs) -> str:
        raise NotImplementedError("Subclasses must implement 'generate'")

class LargeLanguageModel(BaseModel):
    """Integrates with models/gemini-2.0-flash API (or similar LLM).""" # [cite: 3]
    def __init__(self, name: str, identifier: str, api_client: Optional[Any]):
        super().__init__(name, identifier, api_client)
        if self.api_client is not None and not isinstance(self.api_client, genai.GenerativeModel):
            print(f"Warning: Expected genai.GenerativeModel for {identifier}, but got {type(self.api_client)}. Setting api_client to None.")
            self.api_client = None

    def generate(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1000) -> str:
        """
        Makes an actual API call or uses mock response.
        """
        print(f"[{self.name} ({self.identifier})) Attempting to generate response for prompt: '{prompt[:70]}...'")

        if self.api_client:
            try:
                generation_config = {
                    "temperature": temperature,
                    "max_output_tokens": max_tokens,
                }
                response = self.api_client.generate_content(
                    prompt,
                    generation_config=generation_config
                )
                if response and hasattr(response, 'text') and response.text:
                    return response.text
                elif response and hasattr(response, 'prompt_feedback'):
                    print(f"Prompt feedback from LLM: {response.prompt_feedback}")
                    return f"API did not generate content due to safety concerns."
                return f"API generated an empty response."
            except Exception as e:
                print(f"Error calling LLM API for {self.identifier}: {e}")
                return self._mock_generate(prompt)
        else:
            return self._mock_generate(prompt) # Fallback to mock if client is None

    def _mock_generate(self, prompt: str) -> str:
        """Mock response when API is not available or fails."""
        print(f"[{self.name} ({self.identifier})] (MOCK) Generating response for prompt: '{prompt[:70]}...'")
        if "financial performance of Tech Innovations Inc." in prompt and "fiscal year 2024" in prompt:
            return "## User Query Analysis: Tech Innovations Inc. Financial Performance FY2024, Profitability & Growth, Q1 2025 Outlook.\n\n**Information Needs:**\n\n1.  **Financial reports:** We need to identify relevant reports for Tech Innovations Inc. (e.g., 10-K, earnings releases for FY2024).\n2.  **Market data:** Collect recent stock price and analyst outlooks for Q1 2025.\n3.  **Key performance indicators (KPIs):** Focus on revenue growth and profitability margins (e.g., net income, operating margin).\n\n**Initial Analysis Steps:**\n\n1.  **Data Retrieval:** Access TI's FY2024 financial reports and gather recent market data/analyst forecasts.\n2.  **Financial Statement Analysis:** Calculate key profitability and growth metrics from the collected data.\n3.  **Outlook Formulation:** Synthesize market data and analyst expectations for Q1 2025."
        elif "Based on the user's original financial query" in prompt:
            # This mock response for the final LLM synthesis incorporates the LRM's mock output directly
            return """## Financial Performance Analysis: Tech Innovations Inc. (FY2024)

**Overview:** Tech Innovations Inc. (TI) demonstrated strong financial performance in fiscal year 2024, characterized by robust revenue growth and solid profitability.

**Key Highlights (FY2024):**
* **Revenue Growth:** TI reported a significant 15% increase in revenue.
* **Profitability:** The company achieved a net income of $500 million, showcasing strong operational efficiency and healthy profit margins for the period.

**Outlook (Q1 2025):**
The outlook for the first quarter of 2025 remains positive. Analyst consensus projects continued growth, with revenue expected to reach approximately $2.1 billion. This anticipated growth is primarily attributed to upcoming new product launches and strategic market expansion initiatives.

**Conclusion:** Tech Innovations Inc. concluded FY2024 with impressive financial results, particularly in its growth and profitability metrics. The company appears well-positioned for continued positive performance in the immediate future, supported by its strong product pipeline and market strategy."""
        return f"[{self.name}] (MOCK) Generated a general financial response for: {prompt[:50]}..."

class LargeReasoningModel(BaseModel):
    """Integrates with claude-opus-4-20250514 API (or similar LRM)."""
    def __init__(self, name: str, identifier: str, api_client: Optional[Any]):
        super().__init__(name, identifier, api_client)
        if self.api_client is not None and not isinstance(self.api_client, Anthropic):
            print(f"Warning: Expected anthropic.Anthropic for {identifier}, but got {type(self.api_client)}. Setting api_client to None.") # [cite: 7]
            self.api_client = None

    def generate(self, prompt: str, complexity_level: str = "high") -> str:
        """
        Makes an actual API call or uses mock response.
        """ # [cite: 7, 8]
        print(f"[{self.name} ({self.identifier})) Attempting to perform reasoning for prompt: '{prompt[:70]}...' (Complexity: {complexity_level})")

        if self.api_client:
            try:
                messages: List[MessageParam] = [
                    {"role": "user", "content": prompt}
                ]
                response = self.api_client.messages.create(
                    model=self.identifier,
                    max_tokens=1000,
                    messages=messages,
                    temperature=0.1,
                )
                if response and hasattr(response, 'content') and response.content:
                    return "".join(block.text for block in response.content)
                return f"API generated an empty response."
            except Exception as e:
                print(f"Error calling LRM API for {self.identifier}: {e}")
                return self._mock_generate(prompt)
        else:
            return self._mock_generate(prompt) # Fallback to mock if client is None

    def _mock_generate(self, prompt: str) -> str:
        """Mock response when API is not available or fails."""
        print(f"[{self.name} ({self.identifier})] (MOCK) Performing financial reasoning for prompt: '{prompt[:70]}...'")
        if "analyze financial performance of Tech Innovations Inc." in prompt and "fiscal year 2024" in prompt:
            return "Based on FY2024 reports, Tech Innovations Inc. demonstrated robust revenue growth of 15% and a net income of $500M, indicating strong profitability. The outlook for Q1 2025 suggests continued growth, with analyst consensus projecting $2.1B in revenue, driven by new product launches and market expansion."
        elif "recommend investment" in prompt.lower():
            return f"[{self.name}] (MOCK) LRM reasoned: Evaluated market conditions for investment recommendation."
        return f"[{self.name}] (MOCK) LRM performed complex financial reasoning for: {prompt[:50]}..."

# Mock Retrieval Tools (Adapted for Finance)
#
class Memory:
    """Manages short-term and long-term memory."""
    def __init__(self):
        self.short_term_memory: List[str] = []
        self.long_term_memory: List[str] = [] # Could be a more complex structure (e.g., vector store)

    def add_short_term(self, entry: str):
        self.short_term_memory.append(entry)
        # Keep short-term memory limited
        if len(self.short_term_memory) > 10:
            self.short_term_memory.pop(0)

    def add_long_term(self, entry: str):
        self.long_term_memory.append(entry)
        # In a real system, this might involve embeddings and indexing for efficient retrieval

    def retrieve_short_term(self, query: str = "") -> List[str]:
        # Simple retrieval; in real system, would be more sophisticated (e.g., semantic search)
        return [m for m in self.short_term_memory if query.lower() in m.lower()]

    def retrieve_long_term(self, query: str = "") -> List[str]:
        return [m for m in self.long_term_memory if query.lower() in m.lower()]

class VectorDatabase:
    """Mock Vector Database (Adapted for Finance)."""
    def query(self, embedding: List[float], top_k: int = 5) -> List[str]:
        print(f"Querying Vector DB with embedding (simulating financial data search)...")
        # Simulate results related to financial data for "Tech Innovations Inc."
        return ["company_a_financial_statements_vdb", "tech_sector_performance_vdb"]

class SemanticDatabase:
    """Mock Semantic Database (Adapted for Finance)."""
    def query(self, natural_language_query: str) -> List[str]:
        print(f"Querying Semantic DB for: '{natural_language_query[:70]}...' (simulating financial knowledge search)...")
        # Simulate results related to financial regulations or concepts for "Tech Innovations Inc."
        if "regulations" in natural_language_query.lower() or "compliance" in natural_language_query.lower():
            return ["SEC Regulation S-X requirements for financial statements for tech companies.", "GDPR implications for data-driven financial services."]
        elif "investment terms" in natural_language_query.lower() or "metrics" in natural_language_query.lower():
            return ["Explanation of Price-to-Earnings (P/E) Ratio.", "Definition of Return on Equity (ROE) and its relevance in tech.", "Common valuation methodologies for high-growth tech firms."]
        elif "Tech Innovations Inc." in natural_language_query:
            return ["Explanation of Price-to-Earnings (P/E) Ratio.", "financial_concept_1_sdb"] # Adjusted for specific company query
        return ["financial_concept_1_sdb", "market_definition_2_sdb"]

class SearchEngine:
    """Mock Search Engine (e.g., Google, Financial News Sites Adapted)."""
    def search(self, query: str) -> List[str]:
        print(f"Searching external sources for: '{query[:70]}...' (simulating financial web search)...")
        if "stock price apple" in query.lower():
            return ["Current AAPL stock price: $175.50 (as of market close 2024-05-23).", "Apple Inc. (AAPL) Historical Data."]
        elif "inflation rate" in query.lower():
            return ["Current US Inflation Rate: 3.4% (CPI, April 2024).", "Eurozone Inflation Trends Q1 2024."]
        elif "Tech Innovations Inc. FY2024 earnings report" in query or "TI revenue growth" in query:
            return ["Financial web result for 'Tech Innovations Inc. Q4 2024 Earnings Release: Revenue up 15%, Net Income $500M.'", "Financial Times article on TI's strong FY2024 performance."]
        elif "TI stock price outlook Q1 2025" in query:
            return ["Financial web result for 'Analyst consensus for TI Q1 2025 revenue: $2.1B.'", "Reuters: TI stock outlook positive for next quarter."]
        return [f"Financial web result for '{query}' - Article A", f"Financial web result for '{query}' - News Summary B"]

# 2. Retrieval Agents

class RetrievalAgent:
    """Base class for retrieval agents."""
    def __init__(self, name: str):
        self.name = name

    def retrieve(self, query: str, **kwargs) -> List[str]:
        raise NotImplementedError("Subclasses must implement 'retrieve'")

class KnowledgeBaseRetrievalAgent(RetrievalAgent):
    """Retrieves from Vector and Semantic Databases (Adapted for Finance)."""
    def __init__(self, vector_db: VectorDatabase, semantic_db: SemanticDatabase):
        super().__init__("FinancialKnowledgeAgent")
        self.vector_db = vector_db
        self.semantic_db = semantic_db

    def retrieve(self, query: str, query_type: str = "semantic") -> List[str]:
        results = []
        if query_type == "vector":
            # In a real system, 'query' would first be embedded, likely by an embedding model.
            mock_embedding = [0.2]*768 # Placeholder financial embedding
            results.extend(self.vector_db.query(mock_embedding))
        elif query_type == "semantic":
            results.extend(self.semantic_db.query(query))
        return results

class SearchEngineRetrievalAgent(RetrievalAgent):
    """Retrieves from external search engines (Adapted for Finance)."""
    def __init__(self, search_engines: List[SearchEngine]):
        super().__init__("FinancialSearchAgent")
        self.search_engines = search_engines

    def retrieve(self, query: str) -> List[str]:
        all_results = []
        for engine in self.search_engines:
            all_results.extend(engine.search(query))
        return all_results

# 3. Agentic Reasoning and Console (Adapted for Finance)
#
class AgenticFinancialReasoning:
    """Orchestrates LLM, LRM, and Retrieval Agents for Financial Tasks."""
    def __init__(self, llm_general: LargeLanguageModel,
                 lrm_financial: LargeReasoningModel, memory: Memory,
                 kb_agent: KnowledgeBaseRetrievalAgent, se_agent: SearchEngineRetrievalAgent):
        self.llm_general = llm_general
        self.lrm_financial = lrm_financial
        self.memory = memory
        self.kb_agent = kb_agent
        self.se_agent = se_agent
        # We'll use the general LLM as the primary interpreter
        self.current_llm = self.llm_general

    def process_query(self, modified_input: str) -> str:
        """
        This method embodies the core Agentic Reasoning loop for financial tasks.
        """
        print(f"\n [Agentic Financial Reasoning] Processing input using {self.current_llm.identifier} and {self.lrm_financial.identifier}: '{modified_input[:70]}...'") # [cite: 34]
        self.memory.add_short_term(f"User query: {modified_input}")

        # Step 1: Initial LLM interpretation and plan using the general LLM
        llm_system_prompt_initial = "You are an AI financial analyst assistant. Analyze the user query to determine information needs and initial analysis steps." # [cite: 16, 36]
        initial_llm_thought = self.current_llm.generate(f"{llm_system_prompt_initial} User query: {modified_input}")
        self.memory.add_short_term(f"LLM initial thought: {initial_llm_thought}")
        print(f"LLM Initial Thought: {initial_llm_thought[:200]}...") # Truncate for cleaner output

        # Step 2: Determine retrieval needs based on LLM's thought (Financial-specific)
        retrieval_query = ""
        # Keywords updated for finance
        financial_keywords = ["financial data", "market data", "stock price", "earnings report", "profitability", "revenue growth", "outlook"]
        if any(keyword in initial_llm_thought.lower() or keyword in modified_input.lower() for keyword in financial_keywords):
            retrieval_query = modified_input # Use original query or refined retrieval query
            print(f"Determined retrieval need for financial information based on: '{modified_input[:70]}...'") # [cite: 16, 52]

            # Perform Retrieval using Financial Agents
            kb_results = self.kb_agent.retrieve(retrieval_query, query_type="semantic") # Start with semantic for financial concepts/reports
            kb_results.extend(self.kb_agent.retrieve(retrieval_query, query_type="vector")) # Add vector for specific data
            se_results = self.se_agent.retrieve(retrieval_query)
            # Add specific search queries for the scenario
            se_results.extend(self.se_agent.retrieve("Tech Innovations Inc. FY2024 earnings report"))
            se_results.extend(self.se_agent.retrieve("TI stock price outlook Q1 2025"))

            retrieved_info = "\n".join(kb_results + se_results)
            print(f"Retrieved Financial Information: {retrieved_info[:200]}...")
            self.memory.add_short_term(f"Retrieved: {retrieved_info}")
        else:
            retrieved_info = "No specific financial retrieval needed."
            print(retrieved_info)

        # Step 3: Engage LRM (Financial Reasoning Model) if complex reasoning is needed
        lrm_system_prompt = "You are a specialized financial reasoning engine (claude-opus-4-20250514). Analyze the provided financial data and user query to perform complex financial reasoning, such as calculating metrics, identifying trends, and formulating an outlook." # [cite: 16, 56]
        # Keywords updated for financial reasoning tasks
        reasoning_keywords = ["analyze", "evaluate", "compare", "recommend", "outlook", "briefing"]
        if any(keyword in modified_input.lower() for keyword in reasoning_keywords):
            reasoning_input = f"User query: {modified_input}\nInitial LLM thought: {initial_llm_thought}\nRetrieved Information: {retrieved_info}"
            lrm_output = self.lrm_financial.generate(f"{lrm_system_prompt}\n{reasoning_input}")
            self.memory.add_short_term(f"LRM financial reasoning: {lrm_output}")
            print(f"LRM Financial Output: {lrm_output[:200]}...")
        else:
            lrm_output = "No specific complex financial reasoning performed."
            print(lrm_output)

        # Step 4: Final LLM Synthesis (General LLM)
        final_llm_prompt = f"Based on the user's original financial query: '{modified_input}', the initial LLM analysis: '{initial_llm_thought}', the retrieved financial information: '{retrieved_info}', and the LRM's financial reasoning: '{lrm_output}', provide a comprehensive and concise financial performance analysis and outlook for Tech Innovations Inc." # [cite: 18, 59]
        final_response = self.current_llm.generate(final_llm_prompt, max_tokens=1500)
        self.memory.add_long_term(f"Interaction: Query='{modified_input}' | Response='{final_response}'")
        print(f"\n [Final Financial Response] {final_response[:200]}...")
        return final_response

class AgenticFinancialConsole:
    """The top-level console orchestrating user interaction for Financial Analysis."""
    def __init__(self, gemini_client: Optional[genai.GenerativeModel], claude_client: Optional[Anthropic]):
        # Initialize general LLM
        self.llm_general = LargeLanguageModel("Gemini 2.0 Flash (General)", 'gemini-2.0-flash', gemini_client)
        # Initialize Financial Reasoning Model
        self.lrm_financial = LargeReasoningModel("Claude Opus 4 (Financial Reasoning)", 'claude-opus-4-20250514', claude_client)

        self.memory = Memory() # Memory class remains general
        self.vector_db = VectorDatabase() # Mock DB adapted for finance
        self.semantic_db = SemanticDatabase() # Mock DB adapted for finance
        self.search_engines = [SearchEngine()] # Mock Search Engine adapted for finance
        # Use Financial-specific Retrieval Agents
        self.kb_agent = KnowledgeBaseRetrievalAgent(self.vector_db, self.semantic_db)
        self.se_agent = SearchEngineRetrievalAgent(self.search_engines)

        # Use AgenticFinancialReasoning
        self.agentic_reasoning = AgenticFinancialReasoning(
            llm_general=self.llm_general,
            lrm_financial=self.lrm_financial,
            memory=self.memory,
            kb_agent=self.kb_agent,
            se_agent=self.se_agent
        )

    def run_query(self, user_query: str) -> str:
        print(f"\n--- [Agentic Financial Console] Processing User Query: '{user_query}' ---") # [cite: 20, 32]
        parsed_query = user_query.strip()
        print(f"Query Parsed: '{parsed_query}'")

        # System Prompt for the overall financial task
        system_prompt = "You are a helpful AI financial analyst assistant. Your goal is to provide accurate and comprehensive financial analysis and information." # [cite: 20, 35, 97]
        modified_input = f"{system_prompt}\nUser: {parsed_query}"

        # Agentic Reasoning for Financial Query
        compiled_output = self.agentic_reasoning.process_query(modified_input)

        print(f"--- [Agentic Financial Console] Query Complete ---")
        return compiled_output

# Example Usage (Adapted for Financial Query)
if __name__ == "__main__":
    print("Setting up API clients...")
    # --- Gemini API Setup ---
    GOOGLE_API_KEY = None
    if IN_COLAB:
        try:
            GOOGLE_API_KEY = userdata.get('GEMINI')
            print("Successfully retrieved GEMINI_API_KEY from Colab environment variable.")
        except Exception as e:
            print(f"Could not retrieve GEMINI_API_KEY from Colab userdata: {e}. Checking environment variable.")
    if not GOOGLE_API_KEY:
        GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY") # Fallback to environment variable
        if GOOGLE_API_KEY:
            print("Successfully retrieved GOOGLE_API_KEY from environment variable.")
        else:
            print("WARNING: GOOGLE_API_KEY not found. Gemini API calls will be mocked.")

    gemini_client = None
    if GOOGLE_API_KEY:
        try:
            genai.configure(api_key=GOOGLE_API_KEY)
            gemini_client = genai.GenerativeModel('gemini-2.0-flash')
            print("Gemini GenerativeModel ('gemini-2.0-flash') client initialized.")
        except Exception as e:
            print(f"ERROR initializing Gemini client: {e}. Gemini API calls will be mocked.")
    else:
        print("No Google API Key found, Gemini client will not be initialized.")

    # --- Claude API Setup ---
    CLAUDE_API_KEY = None
    if IN_COLAB:
        try:
            CLAUDE_API_KEY = os.environ.get("CLAUDE3_API_KEY")
            print("Successfully retrieved CLAUDE3_API_KEY from Colab userdata.")
        except Exception as e:
            print(f"Could not retrieve CLAUDE3_API_KEY from Colab userdata: {e}. Checking environment variable.")
    if not CLAUDE_API_KEY:
        CLAUDE_API_KEY = os.environ.get("ANTHROPIC_API_KEY") # Fallback to environment variable
        if CLAUDE_API_KEY:
            print("Successfully retrieved ANTHROPIC_API_KEY from environment variable.")
        else:
            print("WARNING: ANTHROPIC_API_KEY not found. Claude API calls will be mocked.")

    claude_client = None
    if CLAUDE_API_KEY:
        try:
            claude_client = Anthropic(api_key=CLAUDE_API_KEY)
            print("Anthropic Claude client initialized.")
        except Exception as e:
            print(f"ERROR initializing Anthropic client: {e}. Claude API calls will be mocked.")
    else:
        print("No Anthropic API Key found, Claude client will not be initialized.")


    # Instantiate the console with the prepared API clients
    # This line creates the main console object that orchestrates everything
    # Pass the Claude model identifier to the console constructor
    console = AgenticFinancialConsole(gemini_client=gemini_client, claude_client=claude_client)

    print('\n')
    print("\nAgenticFinancialConsole initialized. Running your specific financial query...")
    print('\n')

    # --- Running Your Specific Financial Query ---
    your_financial_query = "Analyze the financial performance of Tech Innovations Inc. (Ticker: TI) for the fiscal year 2024, focusing on profitability and revenue growth, and provide a brief outlook for the next quarter." # This is Sarah's specific query

    print(f"\n--- Running Financial Query: {your_financial_query} ---")
    financial_output = console.run_query(your_financial_query)
    print(f"\n--- Financial Analysis Output ---")
    print(financial_output)
    print("--- End of Financial Analysis Output ---")

    # --- Displaying Memory Contents after your query ---
    print("\n-- Current Short-Term Memory ---")
    print(console.agentic_reasoning.memory.retrieve_short_term())

    print('\n')

    print("\n-- Current Long-Term Memory --")
    print(
        console.agentic_reasoning.memory.retrieve_long_term())

    print('\n')

Setting up API clients...
Successfully retrieved GEMINI_API_KEY from Colab environment variable.
Gemini GenerativeModel ('gemini-2.0-flash') client initialized.
Successfully retrieved CLAUDE3_API_KEY from Colab userdata.
Anthropic Claude client initialized.



AgenticFinancialConsole initialized. Running your specific financial query...



--- Running Financial Query: Analyze the financial performance of Tech Innovations Inc. (Ticker: TI) for the fiscal year 2024, focusing on profitability and revenue growth, and provide a brief outlook for the next quarter. ---

--- [Agentic Financial Console] Processing User Query: 'Analyze the financial performance of Tech Innovations Inc. (Ticker: TI) for the fiscal year 2024, focusing on profitability and revenue growth, and provide a brief outlook for the next quarter.' ---
Query Parsed: 'Analyze the financial performance of Tech Innovations Inc. (Ticker: TI) for the fiscal year 2024, focusing on profitability and revenue growth, and provide a br